In [11]:
import os 
import ollama 
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [2]:
# instantiate your key 
HUGGINGFACE_API_KEY = os.environ.get("HUGGINGFACE_API_KEY")

In [3]:
# calling the model through Ollama 
response = ollama.chat(model='gemma:2b', messages = [{'role': 'user', 'content': 'Hello!'}])
print(response['message']['content'])

Hello! How may I assist you today?


In [4]:
# calling the model through langchain 
llm = Ollama(model='gemma:2b')
response = llm.invoke('Hello!')
print(response)

Hello! It's a pleasure to meet you too. How can I assist you today?


In [9]:
# instantiate the embeddings
embeddings = (OllamaEmbeddings(model='mxbai-embed-large'))